#### Test3

1. Preprocessing

In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download necessary NLTK data
nltk.download("stopwords")
nltk.download("wordnet")


# Load training data from file
def preprocess_data_from_file(filepath):
    with open(filepath, "r") as file:
        data = file.read()
    lines = data.strip().split("\n")
    labels = []
    texts = []
    for line in lines:
        label, text = line.split(" ", 1)
        label = int(label.split("__label__")[1])
        labels.append(label)
        texts.append(text)
    return pd.DataFrame({"label": labels, "text": texts})


train_filepath = "train.3270.txt"
df_train = preprocess_data_from_file(train_filepath)

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)

    # Tokenize and remove stop words
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)


df_train["text"] = df_train["text"].apply(preprocess_text)

# Split data into features and labels
X = df_train["text"]
y = df_train["label"]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and pad sequences
max_vocab_size = 20000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_sequence_length)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HUYNGUYEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HUYNGUYEN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2. Define the RNN Model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embedding_dim = 100

model = Sequential()
model.add(
    Embedding(
        input_dim=max_vocab_size,
        output_dim=embedding_dim,
        input_length=max_sequence_length,
    )
)
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,117,377
Trainable params: 2,117,377
Non-trainable params: 0
_________________________________________________________________


3. Train the Model

In [4]:
batch_size = 32
epochs = 10

history = model.fit(
    X_train_padded,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val_padded, y_val),
)

Epoch 1/10
82/82 [==============================] - 4s 43ms/step - loss: -6.5737 - accuracy: 0.5417 - val_loss: -11.1879 - val_accuracy: 0.5229
Epoch 2/10
82/82 [==============================] - 3s 40ms/step - loss: -13.0926 - accuracy: 0.5459 - val_loss: -16.3399 - val_accuracy: 0.5229
Epoch 3/10
82/82 [==============================] - 3s 39ms/step - loss: -17.9304 - accuracy: 0.5459 - val_loss: -21.1895 - val_accuracy: 0.5229
Epoch 4/10
82/82 [==============================] - 3s 39ms/step - loss: -22.4540 - accuracy: 0.5459 - val_loss: -25.9740 - val_accuracy: 0.5229
Epoch 5/10
82/82 [==============================] - 3s 41ms/step - loss: -26.9737 - accuracy: 0.5459 - val_loss: -30.7127 - val_accuracy: 0.5229
Epoch 6/10
82/82 [==============================] - 3s 39ms/step - loss: -31.4826 - accuracy: 0.5459 - val_loss: -35.3748 - val_accuracy: 0.5229
Epoch 7/10
82/82 [==============================] - 3s 41ms/step - loss: -35.7278 - accuracy: 0.5459 - val_loss: -40.0208 - val_acc

4. Evaluate the Model

In [5]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val_padded, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Predict on new data
new_texts = ["This product is great!", "I did not like this book at all."]
new_texts_preprocessed = [preprocess_text(text) for text in new_texts]
new_texts_sequences = tokenizer.texts_to_sequences(new_texts_preprocessed)
new_texts_padded = pad_sequences(new_texts_sequences, maxlen=max_sequence_length)

predictions = model.predict(new_texts_padded)
print(predictions)

Validation Accuracy: 52.29%
1/1 [==============================] - 0s 194ms/step
[[1.]
 [1.]]


#### Test2

In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Download necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")


# Function to preprocess data from text file
def preprocess_data_from_file(filepath):
    with open(filepath, "r") as file:
        data = file.read()
    lines = data.strip().split("\n")
    labels = []
    texts = []
    for line in lines:
        label, text = line.split(" ", 1)
        label = int(label.split("__label__")[1])
        labels.append(label)
        texts.append(text)
    return pd.DataFrame({"label": labels, "text": texts})


# Load training data from file
train_filepath = "train.3270.txt"
df_train = preprocess_data_from_file(train_filepath)

# Initialize stop words, stemmer, and lemmatizer
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def preprocess_text(text, method="lemmatize"):
    # Lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\s+", " ", text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming or Lemmatization
    if method == "stem":
        tokens = [stemmer.stem(word) for word in tokens]
    elif method == "lemmatize":
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)


# Apply text preprocessing
df_train["text"] = df_train["text"].apply(
    lambda x: preprocess_text(x, method="lemmatize")
)

# Check for imbalanced data
print(df_train["label"].value_counts())

# Split data into features and labels
X = df_train["text"]
y = df_train["label"]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Show the first text converted to numerical form
first_text = X_train.iloc[0]
first_text_tfidf = vectorizer.transform([first_text])
print(f"First text: {first_text}")

# Get the feature names (terms) from the TF-IDF vectorizer
feature_names = vectorizer.get_feature_names_out()

# Display the terms with their corresponding TF-IDF scores for the first text
print("Terms and TF-IDF scores for the first text:")
for index, score in zip(first_text_tfidf.indices, first_text_tfidf.data):
    term = feature_names[index]
    print(f"({index}, {term})\t{score}")

# Model selection and training
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Model evaluation on validation set
y_val_pred = model.predict(X_val_tfidf)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HUYNGUYEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HUYNGUYEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HUYNGUYEN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


label
1    1770
2    1500
Name: count, dtype: int64
First text: great basic midi understanding setup student musician institute la wanted learn midi bought book old picture couple section date able tell difference today computer used music fine great explanation easy discription midi message sent use put together midi system great book helped lot
Terms and TF-IDF scores for the first text:
(12900, wanted)	0.10354144891282427
(12628, used)	0.08944348644565366
(12626, use)	0.0831250697453042
(12428, understanding)	0.13080635575538221
(12041, together)	0.10710789930549768
(12036, today)	0.1113371798815985
(11772, tell)	0.09943254464245621
(11661, system)	0.1255186695640299
(11393, student)	0.12327312776395585
(10518, setup)	0.16561761829137742
(10473, sent)	0.12474533255604606
(10412, section)	0.12890431644477274
(9390, put)	0.08655982591735115
(8770, picture)	0.09968460529551991
(8243, old)	0.08613607521190975
(7875, musician)	0.1390846862387611
(7871, music)	0.08627645262107235
(7599, m

#### Test1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Sample Data (For demonstration purposes)
df_train = pd.DataFrame(
    {
        "label": [1, 2, 1, 2],
        "text": [
            "I love this product, it's amazing.",
            "Terrible experience, would not recommend.",
            "Fantastic quality and great service.",
            "Worst product ever, completely useless.",
        ],
    }
)


# Preprocess text function
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r"\W", " ", text)  # Remove punctuation and special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return " ".join(tokens)


# Apply text preprocessing
df_train["text"] = df_train["text"].apply(preprocess_text)

# Split data into features and labels
X = df_train["text"]
y = df_train["label"]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Show the first text converted to numerical form
first_text = X_train.iloc[0]
first_text_tfidf = vectorizer.transform([first_text])
print(f"First text: {first_text}")

# Get the feature names (terms) from the TF-IDF vectorizer
feature_names = vectorizer.get_feature_names_out()

# Display the terms with their corresponding TF-IDF scores for the first text
print("Terms and TF-IDF scores for the first text:")
for index, score in zip(first_text_tfidf.indices, first_text_tfidf.data):
    term = feature_names[index]
    print(f"({index}, {term})\t{score}")

# Model selection and training
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Model evaluation on validation set
y_val_pred = model.predict(X_val_tfidf)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))